<a href="https://colab.research.google.com/github/pratikunterwegs/holeybirds/blob/master/get_elev_indices_hula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python
# coding: utf-8

import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
# get study area bounding box
hula = ee.FeatureCollection("users/pratik_unterwegs/hula_valley");

# get elevation data
dataset = ee.Image('USGS/SRTMGL1_003');
elevation = dataset.select('elevation');

# hula valley 10km buffer
hula_buffer = hula.geometry()

# define start and end date june and dec 2016
start_date = '2016-06-01'
end_date = '2016-12-01'

# clip elevation data
elevation_clip = elevation.clip(hula_buffer);

# get sentinel data for the time range 2016
sentinel = ee.ImageCollection("COPERNICUS/S2")
sentinel_bounds = sentinel.filterBounds(hula_buffer)
sentinel_2016 = sentinel_bounds.filterDate(start_date, end_date)

In [ ]:

# adding a NDVI band
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    return image.addBands([ndvi])

In [ ]:
# map metrics over sentinel
sentinel_ndvi = sentinel_2016.map(add_ndvi)

# get median values
ndvi_median = sentinel_ndvi.select('ndvi').median()

# clip to geometry
ndvi_clip = ndvi_median.clip(hula_buffer)

In [ ]:
# Define an sld style color ramp to apply to the image.
sld_ramp = \
  '<RasterSymbolizer>' + \
    '<ColorMap type="ramp" extended="false" >' + \
      '<ColorMapEntry color="#0D0887" quantity="-1" label="0"/>' + \
      '<ColorMapEntry color="#CC4678" quantity="0" label="50" />' + \
      '<ColorMapEntry color="#F0F921" quantity="0.5" label="100" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>';


# print layers
vis = {'bands': ['elevation']}
Map = geemap.Map(center=[33.0005,33.5], zoom=9)
Map.addLayer(ndvi_clip.sldStyle(sld_ramp), {}, 'ndvi')
Map.addLayer(hula)
Map.addLayerControl()

Map

In [ ]:
# export image to drive
downConfig = {'scale': 30, "maxPixels": 1.0E13, 'driveFolder': 'hula_rasters'}  # scale means resolution.
name = "raster_hula_srtm_30"
# print(name)
task = ee.batch.Export.image(elevation_clip, name, downConfig)
task.start()

In [ ]:
# export image to drive
downConfig = {'scale': 10, "maxPixels": 1.0E13, 'driveFolder': 'hula_rasters'}  # changed scale for sentinel
name = "raster_hula_ndbi"
# print(name)
task = ee.batch.Export.image(ndbi_clip, name, downConfig)
task.start()

# export ndvi to drive
name_ndvi = "raster_hula_ndvi"
# print(name)
task_ndvi = ee.batch.Export.image(ndvi_clip, name_ndvi, downConfig)
task_ndvi.start()